In [1]:
import tensorflow as tf 
import os

The NEU-CLS dataset has 1800 grayscale images of steel surface defects (200×200 pixels) in six classes: rolled-in scale, patches, crazing, pitted surface, inclusion, and scratches.

The dataset is organized into subdirectories per class in separate train/images and validation/images directories. Example: training images might be in .../train/images/scratches/ and similarly for the other classes. 

I would define my steps for training the classification model as follows:

Part 1: Data Processing

    1a. Proper loading of NEU data
    1b. Augment training set and normalize both sets for unified comparison
  
Part 2: Model Creation and Training

    2a. Defining Convolution Neural Net Model
    2b. Trainings with validation loop
  
Part 3: Selecting Optimal Model

    3a. Save best model into project directory

# Part 1: Data Processing

## 1a. Proper loading of NEU data.

For reproducability I am using python's **os** package to establish given user's current working directory. Then joining with location of our train & validation data.

In [6]:
# user dir
current_dir = os.getcwd()

# known filepaths
train_dir = os.path.join(current_dir, "NEU-CLS", "train", "images")
val_dir   = os.path.join(current_dir, "NEU-CLS", "validation", "images")

# unified variables
image_size = (128, 128)
batch_size = 32
epochs = 10
num_classes = 6

## 1b. Augment training set and normalize both sets for unified comparison

For loading my images and labels I am using Keras package to read directories of images. My code uses TF's *ImageDataGenerator()* instead of the basic *image_dataset_from_directory()* for integrated data augmentation such as normalization (*rescale=1. / 255*) and transformations (to the training data only). By augmenting I am expanding the size of the dataset so our model has more images to train on without having to gather any new data. 

- Normalization: All images rescaled to be in range(0,1) by dividing by 225 and resizing to 128x128 resolution. *color_mode='greyscale'* specified so each image has one channel.

- Augmentation: My transformations to the training data includes rotation and reflection (flip), as well as brightness transformation. Limiting range of augmentation to at most 20% for ensuring the image remains usable. No transformations/augmentations made to validation data to avoid leaking val information.

In [ ]:
# Augmenting training images by rotating flipping and altering brightness
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    rotation_range=20, # random rotations
    horizontal_flip=True, # random horizontal flips
    brightness_range=(0.8, 1.2) # random brightness
)

# just normalizing in the case of val images
val_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255) 

# actually generating the image defined from my ImageDataGenerator() function
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=image_size,
    color_mode='grayscale',
    batch_size=batch_size,
    class_mode='categorical'
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=image_size,
    color_mode='grayscale',
    batch_size=batch_size,
    class_mode='categorical'
)

Found 1440 images belonging to 6 classes.
Found 360 images belonging to 6 classes.


# Part 2. Defining Convolution Neural Net Model

## 2a. I will create a basic **Convolutional Neural Network (CNN)** consisting of three blocks and a classification head. 

Block 1, 2, 3: 

> Layer 1. Convolution layer with **ReLu** activation function. *Conv2D()*
>
> Layer 2. Establishing limitations to our batch pool_size. *MaxPooling2D()*
>
> Layer 3. *Dropout()* layer limiting overfitting (also normalizing).

Classification Head:

> Layer 1: *Flatten()* layer to  the feature maps.
>
> Layer 2: *Dense()* layer using **ReLU** as our activation.
> 
> Layer 3: *Dropout()* layer to limit overfitting model.
>
> Layer 4: *Dense()* layer but now using number of types of defects (classes), thus switch to **softmax** activation.

In [10]:
model = tf.keras.models.Sequential([
    # Block 1
    tf.keras.layers.Conv2D(32, (3, 3), activation= 'relu', input_shape= (128, 128, 1)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Dropout(0.25),
    
    # Block 2
    tf.keras.layers.Conv2D(64, (3, 3), activation= 'relu'),
    tf.keras.layers.MaxPooling2D((2,2)),
    tf.keras.layers.Dropout(0.25),
    
    # Block 3
    tf.keras.layers.Conv2D(128, (3, 3), activation= 'relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Dropout(0.25),
    
    # Classification head
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation= 'relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(num_classes, activation= 'softmax')
])

c:\ProgramData\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Finally, our last block will compile our established CNN. Since there are multiple different classifications possible for image, I will compile model with **Adam** as our optimizer and measure by categorical cross-entropy loss.

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 126, 126, 32)   │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 61, 61, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 28, 28, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 14, 14, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 14, 14, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │     3,211,392 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 6)              │           774 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,304,838 (12.61 MB)

 Trainable params: 3,304,838 (12.61 MB)

 Non-trainable params: 0 (0.00 B)